In [17]:
import time,requests,json,random
import pandas as pd
import datetime as dt


In [18]:
class StockCrawling:
    req = None
    db = None

    def __init__(self):
        self.req = requests.session()

    def __del__(self):
        self.req.close()

    def show_realtime(self, *stock_id):
        twse_url = 'http://mis.twse.com.tw/stock/api/getStockInfo.jsp'
        timestamp = int(time.time() * 1000)
        channels = '|'.join('tse_{}.tw'.format(target_tse) for target_tse in stock_id)
        query_url = '{}?&ex_ch={}&json=1&delay=0&_={}'.format(twse_url, channels, timestamp)

        headers = {'Accept-Language': 'zh-TW',
                   'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.81 Safari/537.36',
                   }

        self.req.get('http://mis.twse.com.tw/stock/index.jsp', headers=headers)
        response = self.req.get(query_url)
        if response.text.strip() == '':
            return None

        # d: 日期, h: 最高, l: 最低, c:代號, n: 名稱, t:時間, o: 開盤, v: 交易量, z: 成交價
        # load JSON from TSE
        #print(response.text)
        content = json.loads(response.text)
        self.req.cookies.clear()
        return content['msgArray']

In [27]:
if __name__ == '__main__':
    # To ill stock IDs that you want.
    data = pd.read_csv("./class.csv")
    stock_ids = [ str(i) for i in data['代碼'].to_list() ]
    #print(stock_ids)
    crawling = StockCrawling()
    try:
        while True:
            now = dt.datetime.now()
            #if now < STOP_TIME:
            data = crawling.show_realtime(*stock_ids)
            records = []
            for s_dict in data:
                try:
                    
                    _dt = dt.datetime.strptime(s_dict['d'], "%Y%m%d").date().strftime('%Y-%m-%d')
                    _t = dt.datetime.strptime(s_dict['t'], '%H:%M:%S').time().strftime('%H:%M:%S')
                    # records.append([_dt, s_dict['c'], s_dict['n'],
                    #             float(s_dict['z']), float(s_dict['h']), float(s_dict['l']), float(s_dict['o']),
                    #             int(s_dict['v'])])
                    if(s_dict['z']!='-'):
                        s_dict['z'] = float(s_dict['z'])
                    else:
                        s_dict['z'] = float('NaN')
                    if(s_dict['tv']!='-'):
                        s_dict['tv'] = float(s_dict['tv'])
                    else:
                        s_dict['tv'] = float('NaN')
                    if(s_dict['h']!='-'):
                        s_dict['h'] = float(s_dict['h'])
                    else:
                        s_dict['h'] = float('NaN')
                    if(s_dict['l']!='-'):
                        s_dict['l'] = float(s_dict['l'])
                    else:
                        s_dict['l'] = float('NaN')
                    if(s_dict['o']!='-'):
                        s_dict['o'] = float(s_dict['o'])
                    else:
                        s_dict['o'] = float('NaN')
                    if(s_dict['v']!='-'):
                        s_dict['v'] = float(s_dict['v'])
                    else:
                        s_dict['v'] = float('NaN')

                    records.append([_dt, _t, s_dict['c'], s_dict['n'], s_dict['z'], s_dict['tv'],
                    s_dict['h'], s_dict['l'], float(s_dict['y']), float(s_dict['o']), int(s_dict['v'])])

                
                except ValueError as _e:
                    print("ValueErr : " + str(_e))
            print(records)
            _sleep_time = random.randint(2, 4)
                # sleep
            time.sleep(_sleep_time)
            print("====== " + time.strftime("%H:%M:%S") + " ======")
    finally:
        print("ERROR")
        del crawling


['9101', '9102', '9103', '9104', '9902', '9904', '9905', '9906', '9907', '9908', '9910', '9911', '9912', '9914', '9915', '9917', '9918', '9919', '9921', '9922', '9924', '9925', '9926', '9927', '9928', '9929', '9930', '9931', '9933', '9934', '9935', '9936', '9937', '9938', '9939', '9940', '9941', '9942', '9943', '9944', '9945', '1701', '1702', '1704', '1707', '1708', '1709', '1710', '1711', '1712', '1713', '1714', '1715', '1716', '1717', '1718', '1720', '1721', '1722', '1723', '1724', '1725', '1726', '1727', '1729', '1730', '1731', '1732', '1733', '1734', '1735', '1736', '1301', '1303', '1304', '1305', '1306', '1307', '1308', '1309', '1310', '1311', '1312', '1313', '1314', '1315', '1316', '1319', '1321', '1323', '1324', '1325', '1326', '2101', '2102', '2103', '2104', '2105', '2106', '2107', '2108', '2109', '1101', '1102', '1103', '1104', '1107', '1109', '1110', '2201', '2204', '2206', '2207', '2501', '2504', '2505', '2506', '2509', '2511', '2512', '2514', '2515', '2516', '2520', '2521',

JSONDecodeError: Expecting value: line 31 column 1 (char 30)